In [ ]:
"""
Cubic Spline Interpolation Implementation for HW01 - Task 2
Author: Solution for Computational Physics Assignment
Date: February 2026

This script implements cubic spline interpolation to generate 100 interpolated points
from 10 original data points, achieving 10x resolution increase.

JupyterLab DISPLAY ONLY VERSION:
  - Plots display inline in JupyterLab
  - NO files saved to disk
  - Clean notebook output

LINEAR INTERPOLATION METHOD:
  - Uses "Sid" - Pure Python formula-based implementation
  - No np.interp() - Manual calculation for comparison purposes

Features:
  - Reads data from file
  - Generates evenly-spaced query points
  - Performs cubic spline interpolation using SciPy
  - Uses Sid for linear interpolation comparison
  - Creates comprehensive visualizations
  - Shows all interpolated points inline
  - PRINTS simplified advantages/disadvantages (title + description only, NO impacts)

File Path: D:/Physics/Computational Physics and Astrophysics/interpolation/HW01_data.txt
"""

%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import os
from scipy.interpolate import CubicSpline


# ============================================================================
# INTERPOLATION FUNCTIONS
# ============================================================================

def create_evenly_spaced_points(start, stop, num):
    """
    Generate evenly-spaced numbers over a specified interval.

    Parameters:
    -----------
    start : float
        Starting value
    stop : float
        Ending value
    num : int
        Number of points to generate

    Returns:
    --------
    ndarray : Array of evenly-spaced values
    """
    return np.linspace(start, stop, num)


def read_data_file(file_path):
    """Read data from tab-separated text file."""
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"File not found: {file_path}")

    x_data = []
    y_data = []

    with open(file_path, 'r') as f:
        lines = f.readlines()

    for line in lines[1:]:  # Skip header
        line = line.strip()
        if not line:
            continue
        values = line.split('\t') if '\t' in line else line.split()
        if len(values) >= 2:
            try:
                x_data.append(float(values[0]))
                y_data.append(float(values[1]))
            except ValueError:
                continue

    return np.array(x_data), np.array(y_data)



def cubic_spline_interpolate(x_data, y_data, x_query):
    """
    Perform cubic spline interpolation on data points.

    Cubic spline interpolation divides the domain into segments and fits
    a cubic polynomial (degree 3) within each segment. The polynomials are
    constrained to be continuous and smooth at the segment boundaries.

    Parameters:
    -----------
    x_data : array-like
        x-coordinates of input data points
    y_data : array-like
        y-coordinates of input data points
    x_query : array-like
        x value(s) where interpolation is desired

    Returns:
    --------
    ndarray : Interpolated y value(s)

    Method:
    -------
    1. Uses scipy.interpolate.CubicSpline to create cubic spline object
    2. Evaluates the spline at query points
    3. Returns smooth interpolated values

    Advantages over linear interpolation:
    - Smoother curves (continuous second derivatives)
    - Better approximation of smooth functions
    - No visible corners or kinks at data points
    """
    # Convert to numpy arrays
    x_data = np.asarray(x_data)
    y_data = np.asarray(y_data)
    x_query = np.asarray(x_query)

    # Sort data if necessary
    if not np.all(x_data[:-1] <= x_data[1:]):
        sort_indices = np.argsort(x_data)
        x_data = x_data[sort_indices]
        y_data = y_data[sort_indices]

    # Create cubic spline interpolator
    # bc_type='natural' means second derivative = 0 at endpoints
    spline = CubicSpline(x_data, y_data, bc_type='natural')

    # Evaluate at query points
    y_interp = spline(x_query)

    return y_interp



def linear_interpolate_sid(x_data, y_data, x_query):
    """
    Linear interpolation using SID - Pure Python formula-based approach.
    
    This is the manual implementation that calculates interpolated values
    using the linear interpolation formula WITHOUT using np.interp().
    
    Formula:
    --------
    For a query point x between x_i and x_{i+1}:
        y = y_i + (y_{i+1} - y_i) * (x - x_i) / (x_{i+1} - x_i)
    
    Parameters:
    -----------
    x_data : array-like
        x-coordinates of input data points (must be sorted)
    y_data : array-like
        y-coordinates of input data points
    x_query : array-like
        x value(s) where interpolation is desired
    
    Returns:
    --------
    ndarray : Interpolated y value(s) using linear interpolation
    
    Method:
    -------
    1. For each query point, find the two surrounding data points
    2. Apply linear interpolation formula manually
    3. Handle edge cases (extrapolation at boundaries)
    """
    # Convert to numpy arrays for consistency
    x_data = np.asarray(x_data)
    y_data = np.asarray(y_data)
    x_query = np.asarray(x_query)
    
    # Ensure x_data is sorted
    if not np.all(x_data[:-1] <= x_data[1:]):
        sort_indices = np.argsort(x_data)
        x_data = x_data[sort_indices]
        y_data = y_data[sort_indices]
    
    # Initialize result array
    y_interp = np.zeros_like(x_query, dtype=float)
    
    # Process each query point individually
    for idx, x in enumerate(x_query):
        # Handle extrapolation below range
        if x <= x_data[0]:
            y_interp[idx] = y_data[0]
            continue
        
        # Handle extrapolation above range
        if x >= x_data[-1]:
            y_interp[idx] = y_data[-1]
            continue
        
        # Find the interval containing x
        # We need to find i such that x_data[i] <= x < x_data[i+1]
        for i in range(len(x_data) - 1):
            if x_data[i] <= x <= x_data[i + 1]:
                # Found the interval! Apply linear interpolation formula
                x0 = x_data[i]
                x1 = x_data[i + 1]
                y0 = y_data[i]
                y1 = y_data[i + 1]
                
                # Linear interpolation formula:
                # y = y0 + (y1 - y0) * (x - x0) / (x1 - x0)
                y_interp[idx] = y0 + (y1 - y0) * (x - x0) / (x1 - x0)
                break
    
    return y_interp

